In [67]:
from BS_simul import DeltaHedging

In [62]:
import matplotlib.pyplot as plt

In [63]:
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 1.0   # Time to maturity (1 year)
r = 0.05  # Risk-free rate
sigma = 0.2 # Volatility

In [64]:
delta_hedging = DeltaHedging(S0, K, T, r, sigma)


In [65]:
stock_paths = delta_hedging.simulate_stock_path(n_simulations=1)

In [66]:
stock_paths

array([[100.        ,  99.34124369,  98.70106359, ...,  77.53657539,
         77.32616395,  77.20844869]])

In [69]:
pip install gym

  Using cached gym-0.26.2-py3-none-any.whl
  Using cached cloudpickle-3.1.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl.metadata (1.0 kB)
Using cached cloudpickle-3.1.0-py3-none-any.whl (22 kB)
Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [70]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Create the CartPole environment
env = gym.make("CartPole-v1")
n_actions = env.action_space.n
state_size = env.observation_space.shape[0]

# Hyperparameters
learning_rate = 0.001
discount_factor = 0.99
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
batch_size = 64
memory_size = 10000
num_episodes = 500

# Replay buffer for experience replay
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []

    def add(self, state, action, reward, next_state, done):
        if len(self.buffer) >= self.capacity:
            self.buffer.pop(0)
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        return [self.buffer[i] for i in indices]

    def size(self):
        return len(self.buffer)

# Build the Q-network
def build_model():
    model = Sequential([
        Dense(24, activation="relu", input_shape=(state_size,)),
        Dense(24, activation="relu"),
        Dense(n_actions, activation="linear")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    return model

# Initialize the Q-network and target network
q_network = build_model()
target_network = build_model()
target_network.set_weights(q_network.get_weights())

# Initialize the replay buffer
replay_buffer = ReplayBuffer(memory_size)


/Users/mcbookairdebat/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mcbookairdebat/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [76]:
for episode in range(num_episodes):
    state = env.reset()
    if isinstance(state, tuple):
        state = state[0]  # Adjust for gym reset() returning a tuple in newer versions
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    while True:
        # Epsilon-greedy action selection
        if np.random.rand() <= epsilon:
            action = np.random.choice(n_actions)
        else:
            q_values = q_network.predict(state, verbose=0)
            action = np.argmax(q_values[0])

        # Take action in the environment
        next_state, reward, done, _ = env.step(action)  # Ensure compatibility with gym's step()
        
        if isinstance(next_state, tuple):
            next_state = next_state[0]  # Adjust for gym returning tuples
        next_state = np.reshape(next_state, [1, state_size])
        total_reward += reward

        # Store the transition in the replay buffer
        replay_buffer.add(state, action, reward, next_state, done)

        state = next_state

        # Train the Q-network using experience replay
        if replay_buffer.size() >= batch_size:
            batch = replay_buffer.sample(batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)

            states = np.vstack(states)
            next_states = np.vstack(next_states)
            actions = np.array(actions)
            rewards = np.array(rewards)
            dones = np.array(dones, dtype=np.float32)

            # Compute target Q-values
            target_q_values = q_network.predict(next_states, verbose=0)
            targets = rewards + (1 - dones) * discount_factor * np.amax(target_q_values, axis=1)

            # Update Q-network
            q_values = q_network.predict(states, verbose=0)
            for i, action in enumerate(actions):
                q_values[i][action] = targets[i]

            q_network.fit(states, q_values, epochs=1, verbose=0)

        if done:
            break

    # Update the target network
    target_network.set_weights(q_network.get_weights())

    # Update epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}, Epsilon: {epsilon:.2f}")

# Close the environment
env.close()


ValueError: too many values to unpack (expected 4)